# ICU Mortality Model - Feature Engineering

This notebook loads the ICU cohort and creates hourly wide dataset for the first 24 hours of ICU stay.

## Objective
- Load ICU cohort from 01_cohort.ipynb
- Use pyCLIF to extract features from CLIF tables
- Create hourly wide dataset for the first 24 hours
- Filter to encounters with complete 24-hour data
- Save features for modeling

## Feature Sources
- **Vitals**: All vital_category values
- **Labs**: All lab_category values
- **Patient Assessments**: GCS_total, RASS
- **Respiratory Support**: Mode, FiO2, PEEP, ventilator settings (with one-hot encoding)
- **Medications**: All vasoactives and sedatives

## Setup and Configuration

In [1]:
import sys
import os
sys.path.append(os.path.join('..', 'src'))

import pandas as pd
import numpy as np
from pyclif import CLIF
from pyclif.utils.wide_dataset import convert_wide_to_hourly
import json
import warnings
warnings.filterwarnings('ignore')

print("=== ICU Mortality Model - Feature Engineering ===")
print("Setting up environment...")

=== ICU Mortality Model - Feature Engineering ===
Setting up environment...


In [2]:
def load_config():
    """Load configuration from config.json"""
    config_path = os.path.join("config_demo.json")
    
    if os.path.exists(config_path):
        with open(config_path, 'r') as file:
            config = json.load(file)
        print("✅ Loaded configuration from config.json")
    else:
        raise FileNotFoundError("Configuration file not found. Please create config.json based on the config_template.")
    
    return config

# Load configuration
config = load_config()
print(f"Site: {config['site']}")
print(f"Data path: {config['clif2_path']}")
print(f"File type: {config['filetype']}")

✅ Loaded configuration from config.json
Site: MIMIC
Data path: /Users/sudo_sage/Documents/work/mimic_demo
File type: parquet


In [3]:
# Initialize pyCLIF
clif = CLIF(
    data_dir=config['clif2_path'],
    filetype=config['filetype'],
    timezone="US/Eastern"
)

print("✅ pyCLIF initialized successfully")

CLIF Object Initialized.
✅ pyCLIF initialized successfully


## Load ICU Cohort

In [4]:
# Load ICU cohort from 01_cohort.ipynb
cohort_path = os.path.join('output', 'intermitted', 'icu_cohort.csv')

if os.path.exists(cohort_path):
    cohort_df = pd.read_csv(cohort_path)
    
    # Convert datetime columns
    datetime_cols = ['start_dttm', 'hour_24_start_dttm', 'hour_24_end_dttm']
    for col in datetime_cols:
        cohort_df[col] = pd.to_datetime(cohort_df[col])
    
    print(f"✅ Loaded ICU cohort: {len(cohort_df)} hospitalizations")
    print(f"Mortality rate: {cohort_df['disposition'].mean():.3f}")
    print(f"Time range: {cohort_df['start_dttm'].min()} to {cohort_df['start_dttm'].max()}")
    
else:
    raise FileNotFoundError(f"Cohort file not found at {cohort_path}. Please run 01_cohort.ipynb first.")

# Display sample
print("\nSample cohort records:")
print(cohort_df.head())

✅ Loaded ICU cohort: 89 hospitalizations
Mortality rate: 0.090
Time range: 2110-04-11 20:52:22+00:00 to 2201-12-12 01:11:52+00:00

Sample cohort records:
   hospitalization_id                start_dttm        hour_24_start_dttm  \
0            24597018 2157-11-21 00:18:02+00:00 2157-11-21 00:18:02+00:00   
1            25563031 2110-04-11 20:52:22+00:00 2110-04-11 20:52:22+00:00   
2            20321825 2156-05-01 02:53:00+00:00 2156-05-01 02:53:00+00:00   
3            23473524 2156-05-11 19:49:34+00:00 2156-05-11 19:49:34+00:00   
4            28662225 2156-04-12 21:24:18+00:00 2156-04-12 21:24:18+00:00   

           hour_24_end_dttm  disposition  
0 2157-11-22 00:18:02+00:00            0  
1 2110-04-12 20:52:22+00:00            0  
2 2156-05-02 02:53:00+00:00            0  
3 2156-05-12 19:49:34+00:00            0  
4 2156-04-13 21:24:18+00:00            0  


## Feature Extraction Configuration

In [5]:
# Define feature extraction configuration
print("Configuring feature extraction...")

# Get hospitalization IDs from cohort
cohort_ids = cohort_df['hospitalization_id'].astype(str).unique().tolist()
print(f"Extracting features for {len(cohort_ids)} hospitalizations")

# Define category filters for each table
category_filters = {
    'vitals': [  # Common vital signs
        'heart_rate', 'map', 'respiratory_rate', 'spo2', 'temp_c',
        'weight_kg', 'height_cm'
    ],
    'labs': [  # Common lab values
        "albumin",    "alkaline_phosphatase",    "alt",    "ast",    "basophils_percent",    "basophils_absolute",    "bicarbonate",    "bilirubin_total",    "bilirubin_conjugated",    "bilirubin_unconjugated",
    "bun",
    "calcium_total",    "calcium_ionized",    "chloride",    "creatinine",    "crp",    "eosinophils_percent",
    "eosinophils_absolute",    "esr",    "ferritin",    "glucose_fingerstick",    "glucose_serum",    "hemoglobin",    "phosphate",    "inr",    "lactate",    "ldh",
    "lymphocytes_percent",    "lymphocytes_absolute",    "magnesium",    "monocytes_percent",    "monocytes_absolute",    "neutrophils_percent",    "neutrophils_absolute",
    "pco2_arterial",    "po2_arterial",    "pco2_venous",    "ph_arterial",    "ph_venous",    "platelet_count",    "potassium",    "procalcitonin",
    "pt",    "ptt",    "so2_arterial",    "so2_mixed_venous",    "so2_central_venous",    "sodium",
    "total_protein",    "troponin_i",    "troponin_t",    "wbc"
    ],
    'patient_assessments': [  # Neurological assessments
        'gcs_total', 'rass'
    ],
    'medication_admin_continuous': [  # Vasoactives and sedatives
        "norepinephrine",
    "epinephrine",
    "phenylephrine",
    "angiotensin",
    "vasopressin",
    "dopamine",
    "dobutamine",
    "milrinone",
    "isoproterenol",
    "propofol",
    "dexmedetomidine",
    "ketamine",
    "midazolam",
    "fentanyl",
    "hydromorphone",
    "morphine",
    "remifentanil",
    "pentobarbital",
    "lorazepam"
    ],
    'respiratory_support': [  # All respiratory support categories
        'mode_category', 'device_category', 'fio2'
    ]
}

print("Feature extraction configuration:")
for table, categories in category_filters.items():
    print(f"  {table}: {len(categories)} categories")
    print(f"    {categories[:5]}..." if len(categories) > 5 else f"    {categories}")

Configuring feature extraction...
Extracting features for 89 hospitalizations
Feature extraction configuration:
  vitals: 7 categories
    ['heart_rate', 'map', 'respiratory_rate', 'spo2', 'temp_c']...
  labs: 52 categories
    ['albumin', 'alkaline_phosphatase', 'alt', 'ast', 'basophils_percent']...
  patient_assessments: 2 categories
    ['gcs_total', 'rass']
  medication_admin_continuous: 19 categories
    ['norepinephrine', 'epinephrine', 'phenylephrine', 'angiotensin', 'vasopressin']...
  respiratory_support: 3 categories
    ['mode_category', 'device_category', 'fio2']


## Create Wide Dataset Using pyCLIF

In [7]:
# Create wide dataset for cohort hospitalizations
print("Creating wide dataset using pyCLIF...")


wide_df = clif.create_wide_dataset(
    hospitalization_ids=cohort_ids,
    optional_tables=['vitals', 'labs', 'patient_assessments', 'medication_admin_continuous', 'respiratory_support'],
    category_filters=category_filters,
    save_to_data_location=False  # Keep in memory for processing
)


Creating wide dataset using pyCLIF...
Starting wide dataset creation...
Base tables loaded - Patient: 100, Hospitalization: 275, ADT: 1136
Filtering to specific hospitalization IDs: 89 encounters
Base cohort created with 89 records
Processing vitals: 73187 records
Filtered vitals to categories: ['heart_rate', 'map', 'respiratory_rate', 'spo2', 'temp_c', 'weight_kg', 'height_cm']
Pivoted vitals: 16362 records with 7 category columns
Processing labs: 21810 records
Filtered labs to categories: ['albumin', 'alkaline_phosphatase', 'alt', 'ast', 'basophils_percent', 'basophils_absolute', 'bicarbonate', 'bilirubin_total', 'bilirubin_conjugated', 'bilirubin_unconjugated', 'bun', 'calcium_total', 'calcium_ionized', 'chloride', 'creatinine', 'crp', 'eosinophils_percent', 'eosinophils_absolute', 'esr', 'ferritin', 'glucose_fingerstick', 'glucose_serum', 'hemoglobin', 'phosphate', 'inr', 'lactate', 'ldh', 'lymphocytes_percent', 'lymphocytes_absolute', 'magnesium', 'monocytes_percent', 'monocytes_a

In [16]:
wide_df.to_csv("wide_df.csv", index=False)

## Filter to 24-Hour Window

In [13]:
# Filter wide dataset to 24-hour windows
print("Filtering to 24-hour windows...")

# Merge with cohort to get time windows
wide_df_filtered = pd.merge(
    wide_df,
    cohort_df[['hospitalization_id', 'hour_24_start_dttm', 'hour_24_end_dttm', 'disposition']],
    on='hospitalization_id',
    how='inner'
)

print(f"After merge with cohort: {len(wide_df_filtered)} records")

# Filter events within 24-hour window
wide_df_filtered = wide_df_filtered[
    (wide_df_filtered['event_time'] >= wide_df_filtered['hour_24_start_dttm']) &
    (wide_df_filtered['event_time'] <= wide_df_filtered['hour_24_end_dttm'])
].reset_index(drop=True)

print(f"✅ Filtered to 24-hour windows: {len(wide_df_filtered)} records")
print(f"Hospitalizations with data: {wide_df_filtered['hospitalization_id'].nunique()}")

# Show time window validation
print("\nTime window validation:")
print(f"All events within window: {((wide_df_filtered['event_time'] >= wide_df_filtered['hour_24_start_dttm']) & (wide_df_filtered['event_time'] <= wide_df_filtered['hour_24_end_dttm'])).all()}")
print(f"Average records per hospitalization: {len(wide_df_filtered) / wide_df_filtered['hospitalization_id'].nunique():.1f}")

Filtering to 24-hour windows...
After merge with cohort: 0 records
✅ Filtered to 24-hour windows: 0 records
Hospitalizations with data: 0

Time window validation:
All events within window: True


ZeroDivisionError: division by zero

## Prepare Respiratory Support One-Hot Encoding

In [ ]:
# Prepare respiratory support for one-hot encoding
print("Preparing respiratory support one-hot encoding...")

# Check for respiratory support categorical columns
resp_categorical_cols = ['mode', 'device_category']
resp_available_cols = [col for col in resp_categorical_cols if col in wide_df_filtered.columns]

if resp_available_cols:
    print(f"Found respiratory support categorical columns: {resp_available_cols}")
    
    # Show unique values for each categorical column
    for col in resp_available_cols:
        unique_vals = wide_df_filtered[col].dropna().unique()
        print(f"  {col}: {len(unique_vals)} unique values")
        print(f"    Values: {list(unique_vals)[:10]}..." if len(unique_vals) > 10 else f"    Values: {list(unique_vals)}")
        
        # Create one-hot encoded columns
        for val in unique_vals:
            if pd.notna(val) and val != '':
                col_name = f"{col}_{str(val).lower().replace(' ', '_').replace('/', '_').replace('-', '_')}"
                wide_df_filtered[col_name] = (wide_df_filtered[col] == val).astype(int)
                print(f"    Created: {col_name}")
    
    print(f"✅ One-hot encoding completed for respiratory support")
else:
    print("No respiratory support categorical columns found")

# Check total columns after one-hot encoding
print(f"\nTotal columns after one-hot encoding: {len(wide_df_filtered.columns)}")

## Configure Hourly Aggregation

In [ ]:
# Configure hourly aggregation
print("Configuring hourly aggregation...")

# Define columns for different aggregation methods
continuous_cols = []
categorical_cols = []
boolean_cols = []
one_hot_cols = []

# Identify column types
for col in wide_df_filtered.columns:
    if col in ['hospitalization_id', 'patient_id', 'event_time', 'day_number', 'hosp_id_day_key', 
               'hour_24_start_dttm', 'hour_24_end_dttm', 'disposition']:
        continue  # Skip meta columns
    
    # Check if it's a one-hot encoded column (from respiratory support)
    if any(col.startswith(f"{cat}_") for cat in resp_available_cols):
        one_hot_cols.append(col)
    # Check if it's a medication (boolean)
    elif col in category_filters.get('medication_admin_continuous', []):
        boolean_cols.append(col)
    # Check if it's categorical (original respiratory support columns)
    elif col in resp_categorical_cols:
        categorical_cols.append(col)
    # Otherwise, treat as continuous
    else:
        continuous_cols.append(col)

# Configure aggregation
aggregation_config = {
    'min': continuous_cols,
    'max': continuous_cols,
    'mean': continuous_cols,
    'boolean': boolean_cols + one_hot_cols,  # Both medications and one-hot encoded columns
    'last': categorical_cols  # For original categorical columns
}

print(f"Aggregation configuration:")
for method, cols in aggregation_config.items():
    if cols:
        print(f"  {method}: {len(cols)} columns")
        print(f"    Examples: {cols[:5]}..." if len(cols) > 5 else f"    All: {cols}")
    else:
        print(f"  {method}: No columns")

print(f"\n✅ Hourly aggregation configured")

## Convert to Hourly Dataset

In [ ]:
# Convert to hourly dataset
print("Converting to hourly dataset...")

try:
    hourly_df = convert_wide_to_hourly(wide_df_filtered, aggregation_config)
    
    print(f"✅ Hourly dataset created: {len(hourly_df)} rows, {len(hourly_df.columns)} columns")
    print(f"Reduction: {len(wide_df_filtered)} → {len(hourly_df)} rows ({(1-len(hourly_df)/len(wide_df_filtered))*100:.1f}% reduction)")
    print(f"Hospitalizations: {hourly_df['hospitalization_id'].nunique()}")
    print(f"Max nth_hour: {hourly_df['nth_hour'].max()} (≈ {hourly_df['nth_hour'].max():.1f} hours)")
    
    # Show column naming convention
    print("\nColumn naming examples:")
    sample_cols = [col for col in hourly_df.columns if '_' in col and col not in ['hospitalization_id', 'patient_id', 'event_time']][:10]
    for col in sample_cols:
        print(f"  {col}")
    
except Exception as e:
    print(f"❌ Error converting to hourly dataset: {str(e)}")
    import traceback
    traceback.print_exc()
    raise

## Filter to Complete 24-Hour Data

In [ ]:
# Filter to encounters with complete 24-hour data
print("Filtering to encounters with complete 24-hour data...")

# Count hours per hospitalization
hours_per_hosp = hourly_df.groupby('hospitalization_id')['nth_hour'].nunique().reset_index()
hours_per_hosp.columns = ['hospitalization_id', 'total_hours']

print(f"Hours per hospitalization distribution:")
print(hours_per_hosp['total_hours'].describe())

# Filter to hospitalizations with at least 20 hours of data (allowing for some missing hours)
min_hours_threshold = 20
complete_hosp_ids = hours_per_hosp[hours_per_hosp['total_hours'] >= min_hours_threshold]['hospitalization_id'].unique()

print(f"\nHospitalizations with ≥{min_hours_threshold} hours: {len(complete_hosp_ids)} / {len(hours_per_hosp)}")

# Filter hourly dataset
hourly_df_complete = hourly_df[hourly_df['hospitalization_id'].isin(complete_hosp_ids)].copy()

# Filter to first 24 hours only
hourly_df_complete = hourly_df_complete[hourly_df_complete['nth_hour'] < 24].copy()

print(f"✅ Final hourly dataset: {len(hourly_df_complete)} rows")
print(f"Hospitalizations: {hourly_df_complete['hospitalization_id'].nunique()}")
print(f"Hour range: {hourly_df_complete['nth_hour'].min()} to {hourly_df_complete['nth_hour'].max()}")

# Add outcome variable
hourly_df_complete = pd.merge(
    hourly_df_complete,
    cohort_df[['hospitalization_id', 'disposition']],
    on='hospitalization_id',
    how='left'
)

print(f"Mortality rate in final dataset: {hourly_df_complete['disposition'].mean():.3f}")

## Feature Summary and Validation

In [ ]:
# Create feature summary
print("Creating feature summary...")

# Get feature columns (exclude meta columns)
meta_cols = ['hospitalization_id', 'patient_id', 'nth_hour', 'event_time_hour', 'hour_bucket', 'disposition']
feature_cols = [col for col in hourly_df_complete.columns if col not in meta_cols]

print(f"Total features: {len(feature_cols)}")

# Create feature summary
feature_summary = []

for col in feature_cols:
    non_null_count = hourly_df_complete[col].notna().sum()
    null_count = hourly_df_complete[col].isna().sum()
    completeness = (non_null_count / len(hourly_df_complete)) * 100
    
    # Determine feature type
    if col.endswith('_boolean'):
        feature_type = 'boolean'
        unique_vals = hourly_df_complete[col].dropna().unique()
        stats = {'unique_values': len(unique_vals), 'positive_rate': hourly_df_complete[col].mean()}
    elif col.endswith('_min') or col.endswith('_max') or col.endswith('_mean'):
        feature_type = 'continuous'
        stats = {
            'mean': hourly_df_complete[col].mean(),
            'std': hourly_df_complete[col].std(),
            'min': hourly_df_complete[col].min(),
            'max': hourly_df_complete[col].max()
        }
    elif col.endswith('_last'):
        feature_type = 'categorical'
        unique_vals = hourly_df_complete[col].dropna().unique()
        stats = {'unique_values': len(unique_vals), 'most_common': hourly_df_complete[col].mode().iloc[0] if not hourly_df_complete[col].empty else None}
    else:
        feature_type = 'other'
        stats = {}
    
    feature_summary.append({
        'feature': col,
        'type': feature_type,
        'non_null_count': non_null_count,
        'null_count': null_count,
        'completeness_pct': completeness,
        'stats': stats
    })

feature_summary_df = pd.DataFrame(feature_summary)

print(f"\nFeature summary by type:")
print(feature_summary_df.groupby('type').size())

print(f"\nTop 10 most complete features:")
top_complete = feature_summary_df.nlargest(10, 'completeness_pct')
for _, row in top_complete.iterrows():
    print(f"  {row['feature']}: {row['completeness_pct']:.1f}% ({row['type']})")

print(f"\nFeatures with >50% completeness: {(feature_summary_df['completeness_pct'] > 50).sum()}")
print(f"Features with >80% completeness: {(feature_summary_df['completeness_pct'] > 80).sum()}")

## Save Feature Dataset and Summary

In [ ]:
# Save hourly features dataset
print("Saving feature dataset and summary...")

# Save main hourly dataset
hourly_output_path = os.path.join('output', 'intermitted', 'hourly_features_24hr.csv')
hourly_df_complete.to_csv(hourly_output_path, index=False)

print(f"✅ Hourly features saved to: {hourly_output_path}")
print(f"File size: {os.path.getsize(hourly_output_path) / (1024*1024):.1f} MB")
print(f"Shape: {hourly_df_complete.shape}")

# Save feature summary
summary_output_path = os.path.join('output', 'intermitted', 'feature_summary.csv')
feature_summary_df.to_csv(summary_output_path, index=False)

print(f"✅ Feature summary saved to: {summary_output_path}")

# Save high-level statistics
stats = {
    'total_hospitalizations': int(hourly_df_complete['hospitalization_id'].nunique()),
    'total_hourly_records': int(len(hourly_df_complete)),
    'total_features': len(feature_cols),
    'mortality_rate': float(hourly_df_complete['disposition'].mean()),
    'avg_records_per_hosp': float(len(hourly_df_complete) / hourly_df_complete['hospitalization_id'].nunique()),
    'hour_range': [int(hourly_df_complete['nth_hour'].min()), int(hourly_df_complete['nth_hour'].max())],
    'features_by_type': feature_summary_df.groupby('type').size().to_dict(),
    'high_completeness_features': int((feature_summary_df['completeness_pct'] > 80).sum()),
    'processing_info': {
        'original_wide_records': len(wide_df_filtered),
        'hourly_records': len(hourly_df_complete),
        'reduction_pct': float((1 - len(hourly_df_complete) / len(wide_df_filtered)) * 100),
        'min_hours_threshold': min_hours_threshold
    }
}

stats_path = os.path.join('output', 'intermitted', 'feature_stats.json')
with open(stats_path, 'w') as f:
    json.dump(stats, f, indent=2)

print(f"✅ Feature statistics saved to: {stats_path}")

print("\n🎉 Feature engineering completed successfully!")
print(f"\n📊 Final Dataset Summary:")
print(f"  - Hospitalizations: {stats['total_hospitalizations']:,}")
print(f"  - Hourly records: {stats['total_hourly_records']:,}")
print(f"  - Features: {stats['total_features']:,}")
print(f"  - Mortality rate: {stats['mortality_rate']:.3f}")
print(f"  - Hours per patient: {stats['avg_records_per_hosp']:.1f}")
print(f"  - High-quality features (>80% complete): {stats['high_completeness_features']:,}")